# Model serving & deployment testing

## Abstract

This notebook serves the hypertuned model.

TODO: Update

The model to deploy is an XGBoost regressor with the following hyperparameters values:
- `max_depth` = 9
- `learning_rate` = 0.31619
- `n_estimators` = 767
- `subsample` = 1
- `gamma` = 0

With those hyperparameters values frozen, the model has already been trained on the full data available (_the original file `train.csv`_) during the [hypertuning phase](https://6fbe85b20ee1f013-dot-us-west1.notebooks.googleusercontent.com/lab/tree/Demo%20#2/5.%20Model%20hyper-parameter%20tuning%20&%20evaluation.ipynb).
<br>The model has been serialized in Cloud Storage in the __modelling_serializations__ folder.
<br>The model folder path is: `gs://szilard_aliz_sandbox/pierre_tasks/demo2/modelling_serializations_2/20200216_205755/to_deploy`.

Let's deploy this model and test it with an online prediction request feeding some sample input data taken from the original file `test.csv`.

## Serving the model

### Create endpoint

In [15]:
!gcloud ai endpoints create --region="europe-west3" --display-name="demo2_blackfriday"

Using endpoint [https://europe-west3-aiplatform.googleapis.com/]
Waiting for operation [4533578911413436416]...
........done.
Created Vertex AI endpoint: projects/53911330556/locations/europe-west3/endpoints/8731037118215749632.


In [16]:
!gcloud ai endpoints list --region="europe-west3" --filter=display_name="demo2_blackfriday"

ENDPOINT_ID          DISPLAY_NAME
8731037118215749632  demo2_blackfriday


Using endpoint [https://europe-west3-aiplatform.googleapis.com/]


In [17]:
ENDPOINT_ID = 8731037118215749632

In the following, we use the obtained `ENDPOINT_ID`.

### Create model

The model artifact's filename must exactly match `model.joblib`, if Joblib is used. There are other available formats, see [documentation](https://cloud.google.com/vertex-ai/docs/training/exporting-model-artifacts).

In [32]:
DATETIME = "20220607_192111"
HYPERTUNED_MODEL = f"gs://aliz-ml-spec-2022/demo-2/modelling_serializations/{DATETIME}/gamma=0.0__learning_rate=0.1__max_depth=3__n_estimators=100__n_jobs=-1__subsample=1.0__tree_method=hist.joblib"
MODEL_PATH_TO_DEPLOY = f"gs://aliz-ml-spec-2022/demo-2/modelling_serializations/{DATETIME}/to_deploy"
!gsutil cp $HYPERTUNED_MODEL $MODEL_PATH_TO_DEPLOY/model.joblib

Copying gs://aliz-ml-spec-2022/demo-2/modelling_serializations/20220607_192111/gamma=0.0__learning_rate=0.1__max_depth=3__n_estimators=100__n_jobs=-1__subsample=1.0__tree_method=hist.joblib [Content-Type=application/octet-stream]...
/ [0 files][    0.0 B/214.9 KiB]                                                
/ [1 files][214.9 KiB/214.9 KiB]                                                

Operation completed over 1 objects/214.9 KiB.                                    


In [5]:
IMAGE_URI = "eu.gcr.io/aliz-ml-spec-2022-submission/custom_xgb_prediction:v1"

In [19]:
!docker build -f Dockerfile -t $IMAGE_URI ../

#1 [internal] load build definition from Dockerfile
#1 sha256:2afd271f154e2dc94205e2988a25809ea11ba6e3ad7fbd110c1b4852d3ded7a5
#1 transferring dockerfile: 328B 0.0s done
#1 DONE 0.1s

#2 [internal] load .dockerignore
#2 sha256:cb4a54c141a613cbbe818e0bc2f3e5b36a9ead0d31a9363ff139ee6bf090d551
#2 transferring context: 2B done
#2 DONE 0.1s

#3 [internal] load metadata for europe-docker.pkg.dev/vertex-ai/prediction/xgboost-cpu.1-5:latest
#3 sha256:c52b0375e930a8a96ccb5a2c96e6a978a01bf72bab71abde16c79496f800c414
#3 DONE 0.8s

#4 [1/5] FROM europe-docker.pkg.dev/vertex-ai/prediction/xgboost-cpu.1-5:latest@sha256:7a8dc6fa7025bf46d5ac2beb90c9251471455b36c51be3b2664294b1c3b483e2
#4 sha256:4a22e6a9962f98b9b618bf946eae62f174b3ca28b44306197b2ef177c0fc42d9
#4 resolve europe-docker.pkg.dev/vertex-ai/prediction/xgboost-cpu.1-5:latest@sha256:7a8dc6fa7025bf46d5ac2beb90c9251471455b36c51be3b2664294b1c3b483e2 0.0s done
#4 sha256:7a8dc6fa7025bf46d5ac2beb90c9251471455b36c51be3b2664294b1c3b483e2 9.30kB / 9.30

In [20]:
!docker push $IMAGE_URI

The push refers to repository [eu.gcr.io/aliz-ml-spec-2022-submission/custom_xgb_prediction]
030b68ecedcc: Preparing
1afb051c4f95: Preparing
ab3bc3fabdfe: Preparing
d0aedb6d5eb3: Preparing
222fdfc9acec: Preparing
cb94ad8cef9b: Preparing
fe14ec3e4c0d: Preparing
fda5b3738cd0: Preparing
1fb20d2c0d95: Preparing
f083466fbecc: Preparing
014b8136eea2: Preparing
a35598c8b954: Preparing
324fd3615d10: Preparing
fb2e4792cb2c: Preparing
7c60f7654045: Preparing
fecfc6e0e20f: Preparing
28525d8942f3: Preparing
146ae126a971: Preparing
ba7caf3db0b2: Preparing
ba498da253bd: Preparing
a821d737b082: Preparing
76349470a184: Preparing
05b09c5f7909: Preparing
8a746c199e3e: Preparing
7490b0d11af4: Preparing
a41aafb37690: Preparing
9b2383c5a3e6: Preparing
b76da279980a: Preparing
b015bf11dd72: Preparing
ce220209dc04: Preparing
3e7f9792642f: Preparing
62e7d3082c49: Preparing
80c80c341a44: Preparing
d32df68221c0: Preparing
f79c096f8076: Preparing
30b90239f5ee: Preparing
69c583141dd2: Preparing
6f6c1ae51732: Prepa

In [33]:
!gcloud ai models upload --region="europe-west3" --display-name="blackfriday" --container-image-uri=$IMAGE_URI --artifact-uri=$MODEL_PATH_TO_DEPLOY

Using endpoint [https://europe-west3-aiplatform.googleapis.com/]
Waiting for operation [2583520272762011648]...
.................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................done.


In [35]:
!gcloud ai models list --region="europe-west3"

MODEL_ID             DISPLAY_NAME
5307140317135241216  blackfriday
7804386310512181248  blackfriday
6237133640187248640  blackfriday
3647563854449213440  blackfriday


Using endpoint [https://europe-west3-aiplatform.googleapis.com/]


In [36]:
MODEL_ID = 5307140317135241216

### Deploy model

Deployment might take a while.

In [37]:
!gcloud ai endpoints deploy-model $ENDPOINT_ID --region=europe-west3 --model=$MODEL_ID --display-name="demo2_blackfriday" --machine-type=n1-standard-2 --min-replica-count=1 --max-replica-count=1 --traffic-split=0=100

#### Predict with Gcloud

In [ ]:
%%writefile request.json
{
    "instances": [
        [1000004, "P00128942", "M", "46-50", 7, "B", 2, 1, "1", "11.0", "nan"],
        [1000009, "P00113442", "M", "26-35", 17, "C", 0, 0, "3", "5.0", "nan"],
        [1000010, "P00288442", "F", "36-45", 1, "B", 4, 1, "5", "14.0", "nan"]
    ]
}

In [ ]:
# !gcloud ai endpoints predict 7113681902036320256 --region=europe-west3 --json-request=request.json

#### Predict with Python

In [ ]:
from google.cloud import aiplatform


def predict(
    project: str, location: str, instances: list, endpoint: str
):
    aiplatform.init(project=project, location=location)

    endpoint = aiplatform.Endpoint(endpoint)

    prediction = endpoint.predict(instances=instances)
    print(prediction)
    return prediction

predict(
    project="aliz-ml-spec-2022-submission",
    location="europe-west3",
    instances=[
      [1000004, "P00128942", "M", "46-50", 7, "B", 2, 1, "1", "11.0", "nan"],
      [1000009, "P00113442", "M", "26-35", 17, "C", 0, 0, "3", "5.0", "nan"],
      [1000010, "P00288442", "F", "36-45", 1, "B", 4, 1, "5", "14.0", "nan"]
    ],
    endpoint="7113681902036320256"
)

#### Cleanup

In [14]:
!gcloud ai endpoints undeploy-model --region="europe-west3" --deployed-model-id=7113681902036320256

ERROR: (gcloud.ai.endpoints.undeploy-model) argument --region: ENDPOINT must be specified.
Usage: gcloud ai endpoints undeploy-model (ENDPOINT : --region=REGION) --deployed-model-id=DEPLOYED_MODEL_ID [optional flags]
  optional flags may be  --help | --region | --traffic-split

For detailed information on this command and its flags, run:
  gcloud ai endpoints undeploy-model --help


In [ ]:
!gcloud ai endpoints delete 7113681902036320256 --region="europe-west3"

In [15]:
!gcloud ai models delete 6337901681849663488 --region="europe-west3"

Using endpoint [https://europe-west3-aiplatform.googleapis.com/]
ERROR: (gcloud.ai.models.delete) FAILED_PRECONDITION: The Model "projects/53911330556/locations/europe-west3/models/6337901681849663488" is deployed or being deployed at the following Endpoint(s): projects/53911330556/locations/europe-west3/endpoints/7113681902036320256.
